<a href="https://colab.research.google.com/github/msvt37/HW_11_Risky_Business/blob/main/credit_risk_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ensemble Learning

## Initial Imports

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [38]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read the CSV and Perform Basic Data Cleaning

In [39]:
# Load the data
file_path = Path('/content/drive/MyDrive/Colab Notebooks/ClassificationHW/Starter_Code/Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head(15)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,...,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,0.0,0.0,8.0,0.0,1609.0,14.0,w,10204.87,10204.87,355.29,355.29,295.13,60.16,0.0,0.0,0.0,375.35,May-2019,0.0,1.0,Individual,0.0,178.0,38199.0,3.0,4.0,2.0,4.0,3.0,36590.0,...,1.0,0.0,4.0,6.0,4775.0,447.0,77.7,0.0,0.0,93.0,96.0,3.0,3.0,0.0,20.0,1.0,0.0,1.0,2.0,1.0,1.0,10.0,3.0,3.0,2.0,8.0,0.0,0.0,0.0,3.0,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,0.0,0.0,17.0,1.0,18368.0,34.0,w,24293.13,24293.13,873.53,873.53,706.87,166.66,0.0,0.0,0.0,929.09,May-2019,0.0,1.0,Individual,0.0,165.0,228119.0,3.0,3.0,2.0,3.0,4.0,42273.0,...,4.0,9.0,3.0,13.0,13419.0,29502.0,28.4,0.0,0.0,87.0,286.0,1.0,1.0,2.0,1.0,11.0,3.0,7.0,11.0,8.0,13.0,14.0,13.0,18.0,11.0,17.0,0.0,0.0,0.0,6.0,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,0.0,0.0,8.0,0.0,13247.0,21.0,w,19636.79,19636.79,485.44,485.44,363.21,122.23,0.0,0.0,0.0,529.88,May-2019,0.0,1.0,Individual,0.0,0.0,45684.0,2.0,3.0,2.0,3.0,4.0,32437.0,...,0.0,4.0,1.0,6.0,5711.0,2839.0,62.1,0.0,0.0,159.0,156.0,7.0,4.0,0.0,7.0,7.0,6.0,2.0,5.0,2.0,7.0,8.0,5.0,13.0,5.0,8.0,0.0,0.0,0.0,4.0,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,0.0,1.0,10.0,1.0,17996.0,23.0,w,9719.34,9719.34,335.33,335.33,280.66,54.67,0.0,0.0,0.0,353.55,May-2019,0.0,1.0,Individual,0.0,0.0,68784.0,0.0,4.0,1.0,3.0,9.0,50788.0,...,3.0,5.0,4.0,4.0,6878.0,4899.0,75.1,0.0,0.0,100.0,89.0,24.0,9.0,0.0,34.0,0.0,0.0,4.0,6.0,4.0,4.0,17.0,6.0,6.0,6.0,10.0,0.0,0.0,0.0,1.0,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,0.0,1.0,14.0,0.0,9091.0,28.0,w,21614.73,21614.73,484.36,484.36,385.27,99.09,0.0,0.0,0.0,520.39,May-2019,0.0,1.0,Individual,0.0,0.0,177745.0,0.0,1.0,0.0,2.0,14.0,16828.0,...,2.0,0.0,2.0,4.0,13673.0,19699.0,28.6,0.0,0.0,67.0,138.0,11.0,11.0,4.0,11.0,0.0,0.0,4.0,6.0,7.0,11.0,6.0,12.0,18.0,6.0,14.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
5,25000.0,0.1797,634.43,MORTGAGE,90000.0,Source Verified,Mar-2019,low_risk,n,18.49,1.0,1.0,14.0,0.0,23889.0,58.0,w,24577.72,24577.72,584.51,584.51,422.28,162.23,0.0,0.0,0.0,634.43,May-2019,0.0,1.0,Individual,0.0,0.0,568489.0,1.0,4.0,0.0,3.0,24.0,395045.0,...,1.0,2.0,4.0,8.0,40606.0,813.0,95.7,0.0,0.0,198.0,214.0,6.0,6.0,1.0,13.0,0.0,3.0,3.0,5.0,3.0,7.0,36.0,9.0,21.0,5.0,14.0,0.0,0.0,0.0,1.0,93.1,66.7,0.0,0.0,550717.0,418934.0,18800.0,369051.0,N,N
6,20400.0,0.2000,540.48,RENT,51315.0,Source Verified,Mar-2019,low_risk,n,25.63,0.0,2.0,8.0,1.0,2569.0,35.0,w,20040.85,20040.85,495.15,495.15,359.15,136.00,0.0,0.0,0.0,540.48,

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins=50, figsize=(20,15))
plt.show()

In [40]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['home_ownership'])
df['home_ownership'] = le.transform(df['home_ownership'])

le.fit(df['verification_status'])
df['verification_status'] = le.transform(df['verification_status'])

le.fit(df['loan_status'])
df['loan_status'] = le.transform(df['loan_status'])

le.fit(df['application_type'])
df['application_type'] = le.transform(df['application_type'])

le.fit(df['pymnt_plan'])
df['pymnt_plan'] = le.transform(df['pymnt_plan'])

le.fit(df['issue_d'])
df['issue_d'] = le.transform(df['issue_d'])

le.fit(df['initial_list_status'])
df['initial_list_status'] = le.transform(df['initial_list_status'])




In [41]:
# Some columns seem to have no meaningful data...?
drops = df.columns[df.max() == 0]
drops

Index(['pymnt_plan', 'recoveries', 'collection_recovery_fee', 'acc_now_delinq',
       'num_tl_120dpd_2m', 'num_tl_30dpd', 'tax_liens'],
      dtype='object')

In [42]:
df.drop(['recoveries', 'collection_recovery_fee', 'acc_now_delinq',
       'num_tl_120dpd_2m', 'num_tl_30dpd', 'tax_liens', 'issue_d', 'initial_list_status', 'next_pymnt_d'], axis=1, inplace=True)


In [43]:
df['pymnt_plan'].value_counts()

0    68817
Name: pymnt_plan, dtype: int64

In [44]:
df.drop(['pymnt_plan'], axis=1, inplace=True)

In [45]:
df['hardship_flag'].value_counts()
df['debt_settlement_flag'].value_counts()

#both of these columns only have zeros, drop?

N    68817
Name: debt_settlement_flag, dtype: int64

In [47]:
df.drop(['hardship_flag', 'debt_settlement_flag'], axis=1, inplace=True)


## Split the Data into Training and Testing

In [48]:
# Create our features
X = df.drop(columns='loan_status')

# Create our target
y = df['loan_status']

In [58]:
y_test.shape
y_train.shape

(51612,)

In [49]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.00000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,24.666347,16018.845833,16015.919519,970.639473,970.431151,658.748750,311.855687,0.035072,517.202218,0.016188,1.0,0.139660,188.664298,1.634976e+05,1.023047,3.290481,0.843730,1.909644,14.303152,4.324044e+04,69.21191,1.240042,2.622245,6297.491710,55.702472,42431.895360,1.382071,1.687795,2.189023,4.827572,14142.936193,16991.852667,48.330667,0.006830,2.347065,125.468605,175.136841,14.294215,7.179781,1.441882,24.714954,7.688028,0.472485,3.806123,5.648038,5.135156,7.414403,9.500153,8.630818,13.517663,5.653036,12.571995,0.052138,2.219423,95.057627,30.626217,0.125972,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,12.318917,10002.695495,10004.985940,1229.563301,1229.569864,1126.237377,279.786954,1.086720,816.021053,0.134774,0.0,0.346637,1808.457195,1.723699e+05,1.176782,2.965579,0.970543,1.544994,13.972619,4.956130e+04,22.33711,1.471745,2.502608,5702.281757,18.495480,37166.229316,1.543220,2.852664,2.396186,3.145377,15863.878993,20447.276820,28.106287,0.087327,301.312407,56.968853,99.149467,17.417118,6.303066,1.747777,32.130889,6.034861,1.410819,2.464504,3.470941,3.230651,4.581371,7.573846,5.056994,8.026158,3.472679,6.021318,0.390633,1.897432,8.326426,33.631463,0.336732,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,2.350000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.00000,0.000000,0.000000,0.000000,1.000000,200.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [50]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [51]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,application_type,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
39692,10000.0,0.1131,328.86,1,49000.0,1,24.54,0.0,0.0,10.0,1.0,6580.0,22.0,9289.52,9289.52,942.60,942.60,710.48,232.12,0.0,328.86,0.0,1.0,0,2650.0,277872.0,2.0,2.0,1.0,2.0,2.0,49682.0,52.0,1.0,2.0,3674.0,41.0,18000.0,0.0,1.0,0.0,4.0,27787.0,5812.0,38.8,0.0,0.0,125.0,225.0,3.0,2.0,1.0,3.0,17.0,0.0,2.0,3.0,3.0,4.0,8.0,7.0,13.0,3.0,10.0,0.0,2.0,100.0,0.0,1.0,309140.0,56262.0,9500.0,54197.0
37830,21000.0,0.1072,684.74,3,57400.0,0,34.52,0.0,0.0,12.0,0.0,18688.0,17.0,19411.83,19411.83,2029.21,2029.21,1588.17,441.04,0.0,684.74,0.0,1.0,0,0.0,35707.0,0.0,2.0,0.0,2.0,14.0,17019.0,57.0,0.0,0.0,6449.0,36.0,68100.0,2.0,0.0,0.0,2.0,2976.0,47034.0,27.3,0.0,0.0,46.0,68.0,26.0,14.0,0.0,33.0,14.0,0.0,7.0,8.0,9.0,12.0,3.0,10.0,14.0,8.0,12.0,0.0,0.0,100.0,0.0,0.0,98100.0,35707.0,64700.0,30000.0
64531,35000.0,0.0881,1109.90,2,180000.0,0,3.01,0.0,2.0,7.0,0.0,836.0,19.0,32422.34,32422.34,3312.57,3312.57,2577.66,734.91,0.0,1109.90,0.0,1.0,0,0.0,230404.0,0.0,1.0,0.0,0.0,74.0,386.0,2.0,0.0,2.0,836.0,2.0,41700.0,1.0,0.0,2.0,2.0,32915.0,40664.0,2.0,0.0,0.0,116.0,180.0,13.0,13.0,4.0,13.0,0.0,0.0,1.0,1.0,3.0,9.0,2.0,4.0,13.0,1.0,7.0,0.0,0.0,94.7,0.0,0.0,327877.0,1222.0,41500.0,26177.0
28730,20000.0,0.0819,628.49,2,80000.0,1,26.51,0.0,0.0,15.0,0.0,2615.0,30.0,19012.66,19012.66,1247.88,1247.88,987.34,260.54,0.0,628.49,0.0,1.0,0,0.0,81162.0,0.0,5.0,0.0,3.0,13.0,78547.0,62.0,1.0,1.0,1266.0,44.0,31900.0,1.0,2.0,1.0,4.0,5411.0,12634.0,12.9,0.0,0.0,116.0,112.0,7.0,7.0,1.0,30.0,7.0,0.0,3.0,4.0,4.0,5.0,16.0,10.0,13.0,4.0,15.0,0.0,1.0,90.0,0.0,0.0,143541.0,81162.0,14500.0,111641.0
35102,12600.0,0.1356,427.96,1,71000.0,1,22.38,1.0,2.0,7.0,0.0,9073.0,27.0,12025.61,12025.61,846.43,846.43,574.39,272.04,0.0,427.96,0.0,1.0,0,0.0,271478.0,3.0,3.0,3.0,5.0,2.0,24360.0,49.0,0.0,0.0,9073.0,44.0,26500.0,5.0,4.0,4.0,6.0,38783.0,17427.0,34.2,0.0,0.0,202.0,436.0,34.0,2.0,7.0,34.0,2.0,1.0,1.0,1.0,3.0,5.0,14.0,3.0,6.0,1.0,7.0,0.0,4.0,88.9,0.0,0.0,316682.0,33433.0,26500.0,50182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18872,35000.0,0.0881,1109.90,1,400000.0,1,13.79,0.0,1.0,17.0,1.0,7810.0,46.0,34147.06,34147.06,1075.64,1075.64,852.94,222.70,0.0,1109.90,0.0,1.0,0,118.0,485773.0,3.0,7.0,6.0,12.0,3.0,181668.0,80.0,0.0,0.0,81.0,57.0,49300.0,6.0,10.0,5.0,13.0,28575.0,28516.0,0.3,0.0,0.0,138.0,223.0,43.0,3.0,2.0,43.0,3.0,0.0,2.0,4.0,4.0,8.0,27.0,8.0,17.0,4.0,17.0,0.0,7.0,97.6,0.0,1.0,631699.0,189478.0,28600.0,244399.0
6415,16800.0,0.0881,532.76,2,31000.0,2,20.48,0.0,0.0,3.0,0.0,1867.0,15.0,16390.58,16390.58,516.31,516.31,409.42,106.89,0.0,532.76,0.0,1.0,0,0.0,11771.0,1.0,2.0,1.0,1.0,7.0,9904.0,59.0,1.0,3.0,1867.0,58.0,3500.0,0.0,0.0,1.0,4.0,3924.0,1633.0,53.3,0.0,0.0,123.0,148.0,1.0,1.0,0.0,1.0,8.0,0.0

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [52]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [53]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [54]:
# Scale the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

In [22]:
# Calculated the balanced accuracy score


In [20]:
# Display the confusion matrix


In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
y_pred = model.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance



### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=0)
rf_model = rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)


In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [ ]:
# Print the imbalanced classification report
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.